## Auto Word2Vec Model Generator (beta) by for teamD
### 간단 사용법:
### 0. 압축 해제한 디렉터리를 그대로 사용하셔야 합니다.
### 1.  brand_names 에 알고자 하는 브랜드 이름을 리스트 형식으로 입력합니다. (line 10)
### 2. 학습하고자 하는 텍스트의 수를 정합니다. (line 43, out of range 주의, len(text)권장)
### 3.  Word2Vec 학습에서의 옵션을 설정합니다. 
####    'min_count' :   등장 횟수가 n 이하인 단어는 무시
####    'size' :   n차원짜리 벡터스페이스에 embedding
####    'sg' :   0이면 CBOW, 1이면 skip-gram을 사용한다 (skip-gram 권장)
####    'batch_words':  사전을 구축할때 한번에 읽을 단어 수
####    'iter' :  보통 딥러닝에서 말하는 epoch과 비슷한, 반복 횟수
####    'workers' : multiprocessing.cpu_count()                (출처:blog.theeluwin.k)

In [ ]:
## load libraries
import csv
from konlpy.tag import Okt
from gensim.models.word2vec import Word2Vec
import re
okt = Okt()


## input brand names below
brand_names = ["sns_치킨_total_cleaned"]



## load stopword library
library = "./source/lib/JlabMiner_library.csv"
f = open(library, 'r', encoding = "utf-8")
data = csv.reader(f)
next(data)
stop_words = []
for row in data:
    stop_words.append(row[0])
f.close()

for i in range(len(stop_words)):
    stop_words[i] = stop_words[i].strip()
    
    
## begin processing
for brand_name in brand_names:

    #f = open("./source/data/instagram_" + brand_name + "_cleaned2.csv", 'r', encoding = "utf-8")
    f = open("./source/data/" + brand_name + ".csv", "r", encoding = "utf-8")
    data = csv.reader(f)
    f.close
    next(data)

    text = []
    for row in data:
        text.append(str(row[-1]))
        
       
    ## removing stopwords form text
    text_for_training = []
    for i in range(len(text)):
        buffer = []
        for j in range(len(okt.pos(text[i], norm = True, stem = True))):
            buffer.append(okt.pos(text[i], norm = True, stem = True)[j][0])
        for word in buffer:
            if word in stop_words:
                buffer.remove(word)
        text_for_training.append(' '.join(buffer))
    
    
    ## regexp
    for i in range(len(text_for_training)):
        text_for_training[i] = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…0-9》]', '', text_for_training[i])
        
       
    ## string to list in list
    buff = []
    for i in range(len(text_for_training)):
        buff.append(text_for_training[i].split(" "))            

    ## save models   
    wModel = Word2Vec(list(buff), size = 200, window = 3, min_count=3, sg=1, workers = 4) 
    #wModel.save("./source/model/word2vec_" + brand_name + ".model")
    wModel.save("word2vec_치킨_total.model")

In [35]:
len(buff)

45865

## Word2Vec Instruction
### 1. 저장한 모델을 불러옵니다. 
### 2. 아래 명령어를 사용하여 Word2Vec의 기능을 활용할 수 있습니다.
#### 2-1. model.wv.vocab : 모델에 저장된 Vocab를 확인합니다. 
#### 2-2. model.wv.most_similar(" KeyWord ", topn = n ) : KeyWord 와 유사도가 가장 높은 단어를 n 개만큼 보여줍니다.
#### 2-3. model.wv.similarity(" KW1 ", " KW2 ") : KW1 과 KW2 의 유사도를 나타냅니다.
#### 2-4. 이외의 명령어는 링크에서 확인할 수 있습니다 : https://radimrehurek.com/gensim/models/keyedvectors.html

In [2]:
from gensim.models.word2vec import Word2Vec

## 통합 모델을 보려면 total을 입력하세요!!
brand = "치킨_total"
model = Word2Vec.load("./source/model/word2vec_" + brand + ".model")

In [37]:
model.wv.similarity("저렴하다", "비싸다")

0.8469846

In [36]:
len(model.wv.vocab)

12699

In [12]:
for word in ["신라면", "불닭볶음면", "짜파게티", "너구리"]:

    print(model.wv.similarity(word, "맵다"))

0.6212535
0.7723217
0.5868928
0.7389085


### Word2Vec Vector Map (구현중)

In [7]:
word_vectors  = model.wv
vocabs = word_vectors.vocab.keys()
word_vector_list = [word_vectors[v] for v in vocabs]

In [8]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 2)
xys = pca.fit_transform(word_vector_list)
xs = xys[:,0]
ys = xys[:, 1]

In [9]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

plt.figure(figsize = (10, 10))
plt.scatter(xs, ys, marker = "o")
for i, v in enumerate(vocabs):
    if len(v) > 1:
        plt.annotate(v, xy = (xs[i], ys[i]), fontname = "nanumGothic")
